In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Update Feature Monitor, Feature Group IAM, and Feature Group Service Agent



<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_update_feature_monitor_feature_group_iam_and_service_agent.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Ffeature_store%2Fvertex_ai_feature_store_update_feature_monitor_feature_group_iam_and_service_agent.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/vertex_ai_feature_store_update_feature_monitor_feature_group_iam_and_service_agent.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/vertex_ai_feature_store_update_feature_monitor_feature_group_iam_and_service_agent.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

In this tutorial, you learn how to update a feature monitor, configure feature group IAM Policy, and use a Feature Group Service Agent.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore/overview).

The steps performed include:
* Create a feature group, features, and a feature monitor.
* A service account is created for each feature group when configured to use Feature Group Service Agent.
* Get/List feature group API returns the auto-created service account. Users need to use the command `bq add-iam-policy-binding`  to grant `roles/bigquery.dataViewer` to the provided service account.

## Note

This is a Preview release. By using the feature, you acknowledge that you're aware of the open issues and that this preview is provided “as is” under the pre-GA terms of service.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
LOCATION_PARENT = "projects/" + PROJECT_ID + "/locations/" + LOCATION

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Update Feature Monitor, Feature Group IAM, and Feature Group Service Agent

Import libraries.

In [ ]:
from google.cloud.aiplatform_v1beta1.types import feature as feature_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_group as feature_group_pb2
from google.cloud.aiplatform_v1beta1.types import \
    feature_monitor as feature_monitor_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import io as io_pb2

Configure the necessary service clients:

* [FeatureOnlineStoreAdminServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_online_store_admin_service.FeatureOnlineStoreAdminServiceClient)
* [FeatureRegistryServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1beta1.services.feature_registry_service.FeatureRegistryServiceClient)

In [ ]:
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureRegistryServiceClient)
from google.iam.v1 import iam_policy_pb2, policy_pb2

ENDPOINT = LOCATION + "-aiplatform.googleapis.com"
feature_online_store_admin_service_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)
feature_registry_service_client = FeatureRegistryServiceClient(
    client_options={"api_endpoint": ENDPOINT}
)

Generate sample data for this colab.

In [ ]:
DATASET_ID = "test_data"+"_"+LOCATION.replace('-', '_')  # @param {type:"string"}
TABLE_ID = "tableA"  # @param {type:"string"}

!bq mk --project_id={PROJECT_ID} --dataset_id={DATASET_ID} --location={LOCATION}
!bq query --project_id={PROJECT_ID} --nouse_legacy_sql \
"CREATE TABLE {DATASET_ID}.{TABLE_ID} AS (" \
"SELECT * FROM UNNEST(ARRAY<STRUCT<entity_id STRING, feature_timestamp TIMESTAMP, feature1 INT64, feature2 INT64>>[" \
"('test', TIMESTAMP('2024-02-26 08:00:00 UTC'), 10, 20)," \
"('test', TIMESTAMP('2024-02-27 08:00:00 UTC'), 30, 40)," \
"('test', TIMESTAMP('2024-02-28 08:00:00 UTC'), 50, 60)]))"

## Create `FeatureGroup` and `Feature` resources

Create a `FeatureGroup`.

In [ ]:
FEATURE_GROUP_ID = "product_features_unique"
DATA_SOURCE = f"bq://{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

feature_group_config = feature_group_pb2.FeatureGroup(
    big_query=feature_group_pb2.FeatureGroup.BigQuery(
        big_query_source=io_pb2.BigQuerySource(input_uri=DATA_SOURCE),
        entity_id_columns=["entity_id"],
    ),
    service_agent_type="SERVICE_AGENT_TYPE_FEATURE_GROUP",
    description="This is a feature group for testing",
)

create_group_lro = feature_registry_service_client.create_feature_group(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}",
    feature_group_id=FEATURE_GROUP_ID,
    feature_group=feature_group_config,
)
print(create_group_lro.result())

# Verify feature group is created.
feature_group = feature_registry_service_client.get_feature_group(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}"
)
print(feature_group)

Update the IAM policy on the BigQuery Source.
* Note: The policy change typically takes 2 minutes, so this delay may be reduced. See [IAM docs](https://cloud.google.com/iam/docs/access-change-propagation) for more details.

In [ ]:
SERVICE_ACCOUNT = feature_group.service_account_email
!bq add-iam-policy-binding --project_id={PROJECT_ID} --member=serviceAccount:$SERVICE_ACCOUNT --role=roles/bigquery.dataViewer {DATASET_ID}.{TABLE_ID}

# Wait 10 minutes for the newly updated IAM policy binding to become effective.
from time import sleep

sleep(600)

Create `Feature` resources.

In [ ]:
FEATURE_IDS = ["feature1", "feature2"]
create_feature_lros = []
for id in FEATURE_IDS:
    create_feature_lros.append(
        feature_registry_service_client.create_feature(
            parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}",
            feature_id=id,
            feature=feature_pb2.Feature(),
        )
    )
for lro in create_feature_lros:
    print(lro.result())

# Use list to verify the features are created.
feature_registry_service_client.list_features(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}"
)

Create a `FeatureMonitor`.

In [ ]:
FEATURE_MONITOR_ID = "test_feature_monitor"
schedule_config = feature_monitor_pb2.ScheduleConfig(cron="0 12 * * *")
feature_selection_config = feature_monitor_pb2.FeatureSelectionConfig(
    feature_configs=[
        feature_monitor_pb2.FeatureSelectionConfig.FeatureConfig(
            feature_id="feature1", drift_threshold=0.3
        ),
        feature_monitor_pb2.FeatureSelectionConfig.FeatureConfig(
            feature_id="feature2", drift_threshold=0.3
        ),
    ]
)
create_feature_monitor_lro = feature_registry_service_client.create_feature_monitor(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}",
    feature_monitor_id=FEATURE_MONITOR_ID,
    feature_monitor=feature_monitor_pb2.FeatureMonitor(
        schedule_config=schedule_config,
        feature_selection_config=feature_selection_config,
    ),
)
print(create_feature_monitor_lro.result())

# Verify feature monitor is created.
feature_registry_service_client.get_feature_monitor(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}/featureMonitors/{FEATURE_MONITOR_ID}"
)

Update `FeatureMonitor`

In [ ]:
schedule_config = feature_monitor_pb2.ScheduleConfig(cron="0 14 * * *")
update_feature_monitor_lro = feature_registry_service_client.update_feature_monitor(
    feature_monitor=feature_monitor_pb2.FeatureMonitor(
        name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}/featureMonitors/{FEATURE_MONITOR_ID}",
        description="Test description.",
        schedule_config=schedule_config,
    ),
)
print(update_feature_monitor_lro.result())

Configure IAM Policy for a feature group.


In [ ]:
FEATURE_GROUP_RESOURCE_ID = LOCATION_PARENT + "/featureGroups/" + FEATURE_GROUP_ID
# A specific user email or a group email can be used to add members to the IAM Policy.
EXAMPLE_RESOURCE_VIEWER_EMAIL = (
    "google-cloud-eng-fte@google.com"  # @param {type:"string"}
)
EXAMPLE_DATA_VIEWER_EMAIL = "python-sample-owners@google.com"  # @param {type:"string"}
# A Service Account can also be added to the IAM Policy.
EXAMPLE_ADMIN_SERVICE_ACCOUNT = "samples@python-docs-samples-tests.iam.gserviceaccount.com"  # @param {type:"string"}
feature_registry_service_client.set_iam_policy(
    request=iam_policy_pb2.SetIamPolicyRequest(
        resource=FEATURE_GROUP_RESOURCE_ID,
        policy=policy_pb2.Policy(
            bindings=[
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreResourceViewer",
                    members=[f"group:{EXAMPLE_RESOURCE_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.featurestoreDataViewer",
                    members=[f"group:{EXAMPLE_DATA_VIEWER_EMAIL}"],
                ),
                policy_pb2.Binding(
                    role="roles/aiplatform.admin",
                    members=[f"serviceAccount:{EXAMPLE_ADMIN_SERVICE_ACCOUNT}"],
                ),
            ]
        ),
    )
)

Confirm the updated IAM Policy.

In [ ]:
feature_registry_service_client.get_iam_policy(
    request=iam_policy_pb2.GetIamPolicyRequest(resource=FEATURE_GROUP_RESOURCE_ID)
)

Create feature monitor job.

In [ ]:
feature_registry_service_client.create_feature_monitor_job(
    parent=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}/featureMonitors/{FEATURE_MONITOR_ID}"
)

Get feature statistics.

In [ ]:
feature_registry_service_client.get_feature(
    featurestore_service_pb2.GetFeatureRequest(
        name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}/features/feature1",
        feature_stats_and_anomaly_spec=feature_monitor_pb2.FeatureStatsAndAnomalySpec(
            latest_stats_count=5
        ),
    )
)

## Cleaning up
To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the cluster you created in this tutorial.

In [ ]:
# Delete features
for feature_id in FEATURE_IDS:
    feature_registry_service_client.delete_feature(
        name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}/features/{feature_id}"
    )

# Delete feature group
feature_registry_service_client.delete_feature_group(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureGroups/{FEATURE_GROUP_ID}"
)

# Delete test data
!bq rm --project_id={PROJECT_ID} -f {DATASET_ID}.{TABLE_ID}